In [1]:
!pip -q install https://github.com/Lightning-AI/lightning/archive/refs/heads/master.zip -U
from typing import Callable, Tuple
print('\r')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd
import pytorch_lightning as pl
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
import wandb
from tqdm.auto import tqdm
print('\r')
%matplotlib inline

     \ 17.1 MB 29.7 MB/s 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 52.0 MB/s eta 0:00:00




In [14]:
BATCH_SIZE = 128
EPOCHS = 2
LR = 1e-3
class_weight = [0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,]


In [7]:
def accuracy(x,y):
  return (x.argmax(-1) == y).float().mean()

class Model(nn.Module):
    def __init__(self, num_classes, freeze=True):
        super().__init__()
        self.base = torchvision.models.resnet34(pretrained=False)
        self.base.conv1 = nn.Conv2d(1 ,64 ,kernel_size=7, stride=2, padding=3, bias=False)
        self.base.fc = nn.Identity()
        self.bn = nn.BatchNorm1d(512)
        self.linear = nn.Linear(512, num_classes)

        if freeze:
            self.base = self.base.eval()
            for p in self.base.parameters():
                p.requires_grad = False

    def forward(self, x):
        return self.linear(self.bn(self.base(x)))

class LightningModel(pl.LightningModule):
    def __init__(self, model: nn.Module, lr: float, loss_fn: Callable) -> None:
        super().__init__()
        self.model = model
        self.lr = lr
        self.loss_fn = loss_fn
        # self.accuracy = lambda x, y: (x.argmax(-1) == y).float().mean()

    def common_step(
        self,
        batch: Tuple[torch.FloatTensor, torch.LongTensor],
    ) -> Tuple[torch.FloatTensor, torch.FloatTensor]:
        x, y = batch
        logits = self.model(x)
        loss = self.loss_fn(logits, y)
        acc = accuracy(logits, y)
        return loss, acc

    def training_step(self, batch: Tuple[torch.FloatTensor, torch.LongTensor], *args) -> torch.FloatTensor:
        loss, acc = self.common_step(batch)
        self.log("training_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("training_accuracy", acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch: Tuple[torch.FloatTensor, torch.LongTensor], *args) -> None:
        loss, acc = self.common_step(batch)
        self.log("validation_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("validation_accuracy", acc, on_step=True, on_epoch=True, prog_bar=True)

    def configure_optimizers(self) -> torch.optim.Optimizer:
        return torch.optim.AdamW(self.model.parameters(), self.lr)

In [10]:
tran = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                       torchvision.transforms.RandomHorizontalFlip(),
                                       torchvision.transforms.RandomRotation(30),
                                       torchvision.transforms.Normalize((0.1307,),(0.3081,))])
de= torchvision.datasets.MNIST(root ='/data',download=True,transform= tran)
train = torch.utils.data.DataLoader(de, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=4, pin_memory=True)
se= torchvision.datasets.MNIST(root ='/data',train= False,download=True,transform= tran)
test = torch.utils.data.DataLoader(se, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=4, pin_memory=True)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [16]:
model = Model(10)
lightning_model = LightningModel(model, lr=LR, loss_fn=nn.CrossEntropyLoss(weight = torch.tensor(class_weight)))

In [17]:
trainer = pl.Trainer(
    max_epochs=1,
    accelerator="gpu",  # Указываем, что хотим использовать GPU
    devices=1  # Используем 1 GPU
)
trainer.fit(lightning_model, train, test)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Model            | 21.3 M | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
6.2 K     Trainable params
21.3 M    Non-trainable params
21.3 M    Total params
85.138    Total estimated model params size (MB)
4         Modules in train mode
116       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [18]:

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.eval().to(device)

y_preds = []
ys = []
for x, y in tqdm(test):
    y_preds.append(model(x.to(device)).argmax(dim=-1))
    ys.append(y.to(device))

(torch.cat(y_preds) == torch.cat(ys)).float().mean()


  0%|          | 0/78 [00:00<?, ?it/s]

tensor(0.7259, device='cuda:0')